In [1]:
%pip install spacy nltk 

^C
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.11/site-packages/pip/__main__.py", line 24, in <module>
    sys.exit(_main())
             ^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/pip/_internal/cli/main.py", line 79, in main
    return command.main(cmd_args)
           ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/pip/_internal/cli/base_command.py", line 101, in main
    return self._main(args)
           ^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/pip/_internal/cli/base_command.py", line 236, in _main
    self.handle_pip_version_check(options)
  File "/opt/anaconda3/lib/python3.11/site-packages/pip/_internal/cli/req_command.py", line 177, in handle_pip_version_check
    session = self._build_session(
              ^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages

In [2]:
import json
import spacy
import nltk
from nltk import CFG, ChartParser
from nltk.tokenize import word_tokenize

In [3]:
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/safamohammed/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/safamohammed/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
import pandas as pd 
VLMresponses = pd.read_csv("VLM_responses_Baseline.csv")
VLMresponses = VLMresponses[VLMresponses["Accurate"] != "FALSE"]
VLMresponses = VLMresponses.drop(["Why it Failed?", "Accurate"], axis='columns')
VLMresponses = VLMresponses.reset_index(drop=True)
VLMresponses.head()

,UserPrompt,AssistantResponse,JSON
0,I want to view traffic updates.,The user wants to view traffic updates. To com...,"{\n ""task"":""view traffic updates"",\n ""butt..."
1,I want to change the contact name to Jane Do.,To change the contact name to Jane Do using th...,"{\n ""task"":""change contact name to Jane Do"",..."
2,I want to give my answer verbally.,The image shows a mobile device displaying a u...,"{\n ""task"":""answer verbally and submit"",\n ..."
3,Can I maintain my current weight?,"Based on the image, to complete the task of 'C...","{\n ""task"":""maintain current weight"",\n ""b..."
4,Can you save my location?,"To save the user's location, the user should f...","{\n ""task"":""save location"",\n ""buttons"":[\..."


In [358]:
# Sample Input
text = VLMresponses.iloc[30]["AssistantResponse"]
print(text)

jsonForm = VLMresponses.iloc[30]["JSON"]
print(jsonForm)

Based on the image provided, the user can complete the task in 2 steps:

1. Tap the "DOWNLOAD" button to download the article.
2. The downloaded article will be saved in the user's device for future reference.

Please note that the actual sequence of taps may vary depending on the user's device and the specific implementation of the download feature.
{
   "task":"save the featured article",
   "buttons":[
      "DOWNLOAD"
   ]
}


In [359]:
doc = nlp(text)

In [360]:
from spacy import displacy
displacy.render(doc, style="dep")

In [361]:
def extract_task(doc):
    task = None
    for sent in doc.sents: # break down the document into sentences
        verbs = [token for token in sent if token.pos_ == "VERB"] # find all verbs in the sentence
        for verb in verbs: # look through every word inthe sentence
            verb_subtree = [t for t in verb.subtree if t.i >= verb.i] # find all verbs in the subtree of the current verb
            print(f"verb_subtree for {verb.text} with dep {verb.dep_}: {[t.text for t in verb_subtree]}") # print the subtree of the current verb
            if verb.dep_ in ["advcl"]:
                task = " ".join([t.text for t in verb_subtree]) #  subtree includes the verb and all words attached to it in the tree.
               
            if task: # if a task has been found, break out of the loop
                return task
    return task

print(extract_task(doc))

verb_subtree for Based with dep prep: ['Based', 'on', 'the', 'image', 'provided']
verb_subtree for provided with dep acl: ['provided']
verb_subtree for complete with dep ROOT: ['complete', 'the', 'task', 'in', '2', 'steps', ':', '\n\n', '1', '.']
verb_subtree for Tap with dep ROOT: ['Tap', 'the', '"', 'DOWNLOAD', '"', 'button', 'to', 'download', 'the', 'article', '.', '\n']
verb_subtree for download with dep advcl: ['download', 'the', 'article']
download the article


In [362]:
def extract_buttons(doc):
    buttons = set()
    for sent in doc.sents:
        for token in sent:
            button_name = None
            # if (token.lemma_).lower() in ("tap", "press", "click"):
            #     # button_name = None
            #     for child in token.children:
            #         if child.dep_ in ("prep"):
            #             for grandchild in child.children:
            #                 for word in grandchild.subtree:
            #                     if word.text.lower() in ["button", "icon", "field"]:
            #                         break
            #                     if word.dep_ in ("compound", "amod", "nmod", "pobj", "npadvmod"):
            #                         print(f"Adding word to label: {word.text}")
            #                         if button_name is None:
            #                             button_name = word.text
            #                         else:
            #                             button_name += word.text
            #                         button_name += " "  # Add space between words
            #             buttons.add("".join(button_name.strip())) 
            if (token.lemma_).lower() in ("button", "icon", "field", "option"):
                button_phrase = []
                for child in token.children:
                    if child.dep_ in ("nmod", "compound", "amod", "attr", "appos", "poss"):
                        button_phrase.extend([t.text for t in child.subtree])
                        print(f"Adding button phrase {button_phrase}")
                if button_phrase:
                    if button_phrase[0].lower() in ("the", "a", "an"):
                        button_phrase = button_phrase[1:]
                    buttons.add(" ".join(button_phrase).replace("\"", "").replace("'", "").strip())
    return buttons

print(extract_buttons(doc))

Adding button phrase ['DOWNLOAD']
{'DOWNLOAD'}
